In [26]:
import pyodbc
import pandas as pd
from pathlib import Path
from datetime import date
import numpy as np
import win32com.client as win32

In [27]:
if (date.today()- pd.DateOffset(days=1)).weekday() == 6:
    today = (date.today()- pd.DateOffset(days=3)).strftime('%Y-%m-%d')
else:
    today = (date.today()- pd.DateOffset(days=1)).strftime('%Y-%m-%d')
print(today)

start_date = date.today() - pd.DateOffset(days=8)
start_date = start_date.strftime('%Y-%m-%d')
print(start_date)
# start_date  ='2024-05-01'

2024-06-06
2024-05-30


In [28]:
date_range = pd.date_range(start=start_date, end=today)

for single_date in date_range:
    print(single_date.strftime('%Y-%m-%d'))

2024-05-30
2024-05-31
2024-06-01
2024-06-02
2024-06-03
2024-06-04
2024-06-05
2024-06-06


In [29]:
# today = '2024-04-25'

In [30]:
reason_codes = [
    "Hub SB",
    "WWSB",
    "Broker buy",
    "Return from ODM",
    "To be re-programmed",
    "To be sorted",
    "To be qualified",
    "FXN internal SB",
    "Append EM",
    "Navy deal",
    "EOL without any supply",
    "Waiver process",
    "Others"]

### get external data

In [31]:
# connection
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()
print('connect')

# download shortge from SQL


cursor.execute(f"SELECT * FROM OPS.GPS_tbl_ops_shortage_ext where ReportDate >= '{start_date}'")
shortage_ext = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
shortage_ext['HP_PN'] = shortage_ext['HP_PN'].str.replace('\n', ',', regex=False)
shortage_ext['FV'] = shortage_ext['FV'].str.replace('\n', ',', regex=False)

# download PNFV from SQL
cursor.execute("SELECT * FROM OPS.GPS_tbl_ops_PN_FV")
PNFV = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
PNFV = PNFV.rename({'Descr':'FV'})

connect


In [32]:
# group bu commodity and count the len of each group
shortage_ext = shortage_ext[shortage_ext['P1'] != 0]
ext_count = shortage_ext.groupby(['Commodity']).size().reset_index(name='Count')

# CPU need to add supplier
ext_CPU = shortage_ext[shortage_ext['Commodity'] == 'CPU']
ext_CPU['Supplier'] = np.where(ext_CPU['FV'].str.contains('INTEL', case=False), 'INTEL', 'AMD')
ext_CPU_group = ext_CPU.groupby(['Commodity','Supplier']).size().reset_index(name='Count')
ext_CPU_group['Commodity'] = ext_CPU_group['Supplier'] + ' ' +  ext_CPU_group['Commodity']
ext_CPU_group.drop(columns=['Supplier'], inplace=True)

C:\Users\ChHo808\AppData\Local\Temp\ipykernel_14028\3884976303.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ext_CPU['Supplier'] = np.where(ext_CPU['FV'].str.contains('INTEL', case=False), 'INTEL', 'AMD')


### get internal data

In [33]:
# connection
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()
# download shortge from SQL
cursor.execute(f"SELECT * FROM OPS.GPS_view_ops_critical_shortage_overview where [Report Date] >= '{start_date}'")
internal = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
# check if there is empty value in Commodity
if internal.Commodity.isna().any():
    print(internal[internal.Commodity.isna()])
else:
    print('No empty value in Commodity')

No empty value in Commodity


In [34]:
# drop the rows with 'Delete' == 'Y' and group by Commodity
internal = internal[(~(internal['Delete'] == 'Y'))]

#drop internal P1 = 0 and na
internal = internal.dropna(subset=['P1'])
internal = internal[internal['P1'] != 0]


int_counts = internal.groupby(['Commodity']).size().reset_index(name='Count')

# CPU need to add supplier
int_CPU = internal[internal['Commodity'] == 'CPU']
int_CPU['Supplier'] = np.where(int_CPU['FV Des'].str.contains('INTEL', case=False), 'INTEL', 'AMD')
int_CPU_group = int_CPU.groupby(['Commodity','Supplier']).size().reset_index(name='Count')
int_CPU_group['Commodity'] = int_CPU_group['Supplier'] + ' ' +  int_CPU_group['Commodity']
int_CPU_group.drop(columns=['Supplier'], inplace=True)

C:\Users\ChHo808\AppData\Local\Temp\ipykernel_14028\3113042560.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  int_CPU['Supplier'] = np.where(int_CPU['FV Des'].str.contains('INTEL', case=False), 'INTEL', 'AMD')


Special supply format misaligned (in reason_codes)

In [35]:
df_special_supply = internal.copy()
#special supply is non supplier supply
df_special_supply['ETA'] = df_special_supply['ETA'].str.split('\n')
df_special_supply = df_special_supply.explode('ETA')
text = df_special_supply['ETA']

In [36]:
def check_conditions(text):
   # 检查文本是否以 'ETA' 或 'TBD' 开头
   if text is None:
        return False
   
   if text.startswith('ETA') or text.startswith('TBD'):
       return False
   # 检查文本是否包含任何 reason_codes 中的字符串
   for reason in reason_codes:
       if reason in text:
           return True
   return False
# 应用函数到 'ETA' 列
df_special_supply['ETA misaligned'] = df_special_supply['ETA'].apply(check_conditions)
# 打印结果
print(df_special_supply)
df_spsupply_true = df_special_supply[df_special_supply['ETA misaligned'] == True]
df_spsupply_mis = df_spsupply_true[['Report Date', 'Buyer Name', 'Commodity', 'FV Des', 'ETA']]

            Reference ID      Buyer Name Severity Shortage Start Date  \
0    AC20240530141610001  Josephine Chen        R          2024-05-14   
0    AC20240530141610001  Josephine Chen        R          2024-05-14   
0    AC20240530141610001  Josephine Chen        R          2024-05-14   
0    AC20240530141610001  Josephine Chen        R          2024-05-14   
0    AC20240530141610001  Josephine Chen        R          2024-05-14   
..                   ...             ...      ...                 ...   
918  WE20240603132609001     Kylie/Sally        R          2024-06-03   
918  WE20240603132609001     Kylie/Sally        R          2024-06-03   
919  WE20240605115928001     Kylie/Sally        R          2024-06-05   
920  WE20240606133804001     Kylie/Sally        Y          2024-06-06   
921  WE20240606133806002     Kylie/Sally        R          2024-06-06   

      Commodity                                     HP Part Number  \
0    AC ADAPTOR                              L55693-0

In [37]:
shortage_ext.head()

,ODM,Item,Commodity,FV,Platform,P1,Net P2,Net P3,Total Shortage Qty,BT shortage,Working on upside,ReportDate,last FD date,HP_PN,BuyerName
1,WHFXN,4.0,HDD,500GB 7200 SATA,Trinidad,4229.0,0.0,0.0,4229.0,0.0,1079.0,2024-06-05,2050-01-01,"L86041-001,724967-004,L08546-004",JittungmanSupansa(GPSThailand)
2,CEI,LCD228,LCD,16WQXGA AG UWVA EDP NB sRGB LBL DDS 400N 240HZ,Quaker 23C1,234.0,0.0,0.0,234.0,0.0,234.0,2024-06-05,2050-01-01,"N21832-361,N21832-L61,N21832-S61",WangYu-Ting
3,CQIEC,3.0,LCD,13.5WUXGA+ BV UWVA EDPPSR BENTNB sRGB LBL 400N,Sandwalker,105.0,0.0,0.0,105.0,0.0,5.0,2024-06-05,2050-01-01,"M53580-JR1,M53580-2R2",ChenJasmine
6,CQIEC,8.0,LCD,17.3FHD AG UWVA EDPPSR SLIMNB 300N 60HZ,VladRPL6U,799.0,0.0,0.0,799.0,0.0,0.0,2024-06-05,2024-06-13,"L43245-LG2,L43245-3G1,L43245-2G3,L43245-JG2,L4...",JenBella
7,CQIEC,9.0,LCD,17.3HD+ AG SVA EDP SLIMNBZ 250N,Vlad21C1,1710.0,0.0,0.0,1710.0,0.0,0.0,2024-06-05,2024-06-15,"M12358-LG1,M12358-3G1",JenBella


In [38]:
df_spsupply_mis.head()

,Report Date,Buyer Name,Commodity,FV Des,ETA


In [39]:
CQODM = ['CQQCI', 'CQIEC', 'CQWIS', 'TWIEC', 'L_WHFXN', 'CQPCQ', 'MSI', 'PCQ', 'PCP', 'WHFXN_L5', 'PSZ', 'FLH', 'TWQCI', 'SZBYD']
WHODM = ['WHFXN']
KSODM = ['CEI']
THODM = ['THQCI','THIEC']
MXODM = ['IMX']

ODM_list = CQODM + WHODM + KSODM + THODM + MXODM

incorrect_ODM = []
for index in internal.index:
    if internal.loc[index,'ODM'] not in ODM_list:
        incorrect_ODM.append(internal.loc[index,:])
    else:
        pass

incorrect_ODM = pd.DataFrame(incorrect_ODM)
try:
    incorrect_ODM = incorrect_ODM[['Report Date', 'Buyer Name','Commodity','ODM']]
except:
    pass

incorrect_ODM = incorrect_ODM.drop_duplicates()



print(len(incorrect_ODM))

0


In [40]:
Commodity_list = ['AC ADAPTOR','AudioAMP','AudioCodec','BATTERY','CardReader','CHIPSET','CONNECTOR','CPU','EC','eMMC', 'Ethernet IC','GPU','GPU-Graphic Card','HDD','LanChip','LCD','Memory','NIC',
 'ODD','POWERCORDS','PSU','Retimer','SIO','SSS','ThunderBT','TPM','TS','USBIC','VRAM','WEBCAM','WLAN','WWAN']

incorrect_commodity = []
for index in internal.index:
    if internal.loc[index,'Commodity'] not in Commodity_list:
        incorrect_commodity.append(internal.loc[index,:])
    else:
        pass

incorrect_commodity = pd.DataFrame(incorrect_commodity)
print(len(incorrect_commodity))

0


### merge and compare internal with external

In [41]:
df_compare = int_counts[int_counts['Commodity'] !='CPU'].merge(ext_count[ext_count['Commodity'] != 'CPU'], on=['Commodity'], how='outer',suffixes=('_int', '_ext'))
df_compare_CPU = int_CPU_group.merge(ext_CPU_group, on=['Commodity'], how='outer',suffixes=('_int', '_ext'))
df_compare = pd.concat([df_compare, df_compare_CPU], axis=0)

In [42]:
df_mismatch = df_compare[((df_compare['Count_int'].isnull()) & (df_compare['Count_ext'].notnull())) | ((df_compare['Count_int'].notnull()) & (df_compare['Count_ext'].isnull()))]
df_mismatch.loc[:,['Count_int','Count_ext']] = df_mismatch.loc[:,['Count_int','Count_ext']].fillna(0)
df_mismatch.set_index('Commodity', inplace=True)
df_mismatch = df_mismatch.astype(int)
df_mismatch.rename(columns={'Count_int':'Count_internal', 'Count_ext':'Count_external'}, inplace=True)
df_mismatch = df_mismatch[df_mismatch.index != 'Ethernet IC']
df_mismatch

,Count_internal,Count_external
Commodity,,


In [43]:
if (len(df_mismatch) > 0 ) | (len(incorrect_ODM) > 0) | (len(incorrect_commodity) > 0) | (len(df_spsupply_mis) > 0):
    # sent to list
    To_list = 'gps.taiwan.nb.buy-sell@hp.com'
    CC_list = 'spencer.cheng1@hp.com; dustin.hsu1@hp.com'

    # start creating the email
    mail = win32.Dispatch("Outlook.Application").CreateItem(0)
    mail.To = To_list
    mail.CC = CC_list
    mail.Subject = 'Internal/ External Report Misalignment alert' + '<From Report Date: '+ start_date+' to ' + today + '>'

    # Step 3: Highlight only the Commodity values
    try:
        for _ in df_mismatch.index:
            df_mismatch.loc[_, 'Commodity'] = f'<span style="background-color: yellow;">{_}</span>'
        # df_mismatch['Commodity'] = df_mismatch['Commodity'].apply(lambda name: f'<span style="background-color: yellow;">{name}</span>')
        df_mismatch['Count_internal'] = df_mismatch['Count_internal'].apply(lambda name: f'<font color = Red">{name}</font>' if name == 0 else name)
        df_mismatch['Count_external'] = df_mismatch['Count_external'].apply(lambda name: f'<font color = Red">{name}</font>' if name == 0 else name)
        # df_mismatch.rename(columns={'Count_int':'Count_internal', 'Count_ext':'Count_external'},inplace=True)
        df_mismatch = df_mismatch.reindex(columns=['Commodity','Count_internal','Count_external'])
    except:
        pass

    # df_mismatch = df_mismatch.reindex(columns = ['Commodity','Count_int','Count_ext'])
    # Step 4: Convert the DataFrame to HTML, allowing HTML content within cells
    html_compare_2 = df_mismatch.to_html(escape=False, index=False)
    try:
        incorrect_ODM['ODM'] = incorrect_ODM['ODM'].apply(lambda name: f'<font color = Red">{name}</font>')
    except:
        pass
    try:
        incorrect_commodity['Commodity'] = incorrect_commodity['Commodity'].apply(lambda name: f'<font color = Red">{name}</font>')
    except:
        pass
    # message --> use the html content from the previous step 1&2 to 
    signature = "<br><br>Best Regards,<br>Newcomen Project Team"
    message = f"""Hi team,<br><br>Below information is <b><font color = 'blue'>misaligned/ missing</b></font> in external & internal reports from report date: <b><font color = 'blue'>{start_date}</b></font>,
        please help to check and make amendment regardingliy.<br>{html_compare_2}<br><br>
        Below data contains <b><font color = 'blue'>incorrect ODM name</b></font>, please delete the data and upload again with the correct ODM name.<br>{incorrect_ODM.to_html(escape=False, index=False)}<br><br>
        Below data contains <b><font color = 'blue'>incorrect Commodity name</b></font>, data is deleted by project team, please upload the data with correct commodity name.<br>{incorrect_commodity.to_html(escape=False, index=False)}<br><br>
        Below data contains <b><font color = 'blue'>non supplier supply with incorrect format</b></font>, please revise the data with internal shortage tool accordingly.<br>{df_spsupply_mis.to_html(escape=False, index=False)} <br><br>
        {signature}"""

    # make sure email fully initialized, and only write after <body>
    mail.GetInspector 
    index = mail.HTMLbody.find('>', mail.HTMLbody.find('<body')) 

    # generate email
    mail.HTMLBody = message
    mail.display()

else:
    To_list = 'gps.taiwan.nb.buy-sell@hp.com'
    CC_list = 'spencer.cheng1@hp.com; dustin.hsu1@hp.com'

    # start creating the email
    mail = win32.Dispatch("Outlook.Application").CreateItem(0)
    mail.To = To_list
    mail.CC = CC_list
    mail.Subject = 'Internal/ External Report Misalignment alert' + '<From Report Date: '+ start_date+' to ' + today + '>'
    signature = "<br><br>Best Regards,<br>Newcomen Project Team"
    message = f"""Hi team,<br><br>
    No information is <b><font color = 'blue'>misaligned/ missing</b></font> in external & internal reports from report date: <b><font color = 'blue'>{start_date} to {today}</b></font>.
    Thank you.<br><br>{signature}"""

    mail.GetInspector 
    index = mail.HTMLbody.find('>', mail.HTMLbody.find('<body')) 

    # generate email
    mail.HTMLBody = message
    mail.display()
    
    print(f'No error found on {today}')

No error found on 2024-06-06


In [44]:
# connection
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

# download shortge from SQL
cursor.execute(f"SELECT * FROM OPS.GPS_view_ops_critical_shortage_overview where [Report Date] = '{today}'")
shor = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
conn.close

<function Connection.close>